In [1]:
import pandas as pd
from auto_ml import Predictor
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np

/home/rlfo/.local/share/virtualenvs/classifier_automl-HbKVo98X/lib/python3.7/site-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/home/rlfo/.local/share/virtualenvs/classifier_automl-HbKVo98X/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
# Remover os warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv('top50.csv',encoding = "ISO-8859-1")

In [4]:
# Uma idea do top 10 mais popular
mais_popular = data.sort_values(['Popularity'], ascending=False)
mais_popular.head(10)

,Unnamed: 0,Track.Name,Artist.Name,Genre,Beats.Per.Minute,Energy,Danceability,Loudness..dB..,Liveness,Valence.,Length.,Acousticness..,Speechiness.,Popularity
9,10,bad guy,Billie Eilish,electropop,135,43,70,-11,10,56,194,33,38,95
4,5,Goodbyes (Feat. Young Thug),Post Malone,dfw rap,150,65,58,-4,11,18,175,45,7,94
10,11,Callaita,Bad Bunny,reggaeton,176,62,61,-5,24,24,251,60,31,93
14,15,Money In The Grave (Drake ft. Rick Ross),Drake,canadian hip hop,101,50,83,-4,12,10,205,10,5,92
1,2,China,Anuel AA,reggaeton flow,105,81,79,-4,8,61,302,8,9,92
6,7,Ransom,Lil Tecca,trap music,180,64,75,-6,7,23,131,2,29,92
36,37,Otro Trago,Sech,panamanian pop,176,70,75,-5,11,62,226,14,34,91
21,22,Panini,Lil Nas X,country rap,154,59,70,-6,12,48,115,34,8,91
20,21,Piece Of Your Heart,MEDUZA,pop house,124,74,68,-7,7,63,153,4,3,91
19,20,Truth Hurts,Lizzo,escape room,158,62,72,-3,12,41,173,11,11,91


In [5]:
data.sample(2)

,Unnamed: 0,Track.Name,Artist.Name,Genre,Beats.Per.Minute,Energy,Danceability,Loudness..dB..,Liveness,Valence.,Length.,Acousticness..,Speechiness.,Popularity
40,41,Higher Love,Kygo,edm,104,68,69,-7,10,40,228,2,3,88
24,25,bad guy (with Justin Bieber),Billie Eilish,electropop,135,45,67,-11,12,68,195,25,30,89


In [6]:
# Deletar três colunas que não nos interessa, o ID, o nome da música, e o nome do artista
data = data.drop(columns=['Unnamed: 0','Track.Name','Artist.Name','Genre'])

In [7]:
data.sample(2)

,Beats.Per.Minute,Energy,Danceability,Loudness..dB..,Liveness,Valence.,Length.,Acousticness..,Speechiness.,Popularity
23,92,78,80,-4,44,80,266,36,4,91
9,135,43,70,-11,10,56,194,33,38,95


In [8]:
data.sample(2)

,Beats.Per.Minute,Energy,Danceability,Loudness..dB..,Liveness,Valence.,Length.,Acousticness..,Speechiness.,Popularity
24,135,45,67,-11,12,68,195,25,30,89
49,95,79,75,-6,7,61,206,21,12,82


In [9]:
def colunms_to_description(columns_name, class_column):
    columns_descriptions = {}
    for column in columns_name:
        if(column != class_column):
            columns_descriptions[column] = 'categorical'
        else:
            columns_descriptions[column] = 'output' 
    return columns_descriptions
            
    

In [10]:
def split_data_frame(data, porcentage=0.8):
    train_len = int(len(data)*porcentage)
    df1 = data.iloc[:, :train_len]
    df2 = data.iloc[:, len(data)-train_len:]
    return df1, df2

In [11]:
column_descriptions = colunms_to_description(list(data.columns),'Popularity')

In [12]:
# Modelo de columns_description utilizando dataframe pandas
column_descriptions

{'Beats.Per.Minute': 'categorical',
 'Energy': 'categorical',
 'Danceability': 'categorical',
 'Loudness..dB..': 'categorical',
 'Liveness': 'categorical',
 'Valence.': 'categorical',
 'Length.': 'categorical',
 'Acousticness..': 'categorical',
 'Speechiness.': 'categorical',
 'Popularity': 'output'}

In [13]:
def split_data_frame(X, y, test_size = 0.1, random_state = 42):
        
    np.random.seed(random_state)
    
    msk = np.random.rand(len(y)) < 1-test_size
    
    X_train=X[msk]
    X_test=X[~msk]
    return (X_train, X_test)

In [14]:
df_train, df_test = split_data_frame(X=data, y=data['Popularity'])

In [15]:
df_train.sample(2)

,Beats.Per.Minute,Energy,Danceability,Loudness..dB..,Liveness,Valence.,Length.,Acousticness..,Speechiness.,Popularity
4,150,65,58,-4,11,18,175,45,7,94
45,88,62,66,-2,58,44,182,7,5,88


In [16]:
df_test.sample(2)

,Beats.Per.Minute,Energy,Danceability,Loudness..dB..,Liveness,Valence.,Length.,Acousticness..,Speechiness.,Popularity
43,136,40,90,-9,6,35,198,5,13,84
11,96,71,82,-4,15,38,185,28,7,86


In [17]:
ml_predictor = Predictor(type_of_estimator='regressor', column_descriptions=column_descriptions)
ml_predictor.train(df_train,model_names=['DeepLearningRegressor'],ml_for_analytics=True)

Welcome to auto_ml! We're about to go through and make sense of your data using machine learning, and give you a production-ready pipeline to get predictions with.

If you have any issues, or new feature ideas, let us know at http://auto.ml
You are running on version 2.9.9
Now using the model training_params that you passed in:
{}
After overwriting our defaults with your values, here are the final params that will be used to initialize the model:
{'epochs': 1000, 'batch_size': 50, 'verbose': 2}


Using TensorFlow backend.


Running basic data cleaning
Performing feature scaling
Fitting DataFrameVectorizer
Now using the model training_params that you passed in:
{}
After overwriting our defaults with your values, here are the final params that will be used to initialize the model:
{'epochs': 1000, 'batch_size': 50, 'verbose': 2}


********************************************************************************************
About to fit the pipeline for the model DeepLearningRegressor to predict Popularity
Started at:
2020-01-08 13:18:53

We will stop training early if we have not seen an improvement in validation accuracy in 25 epochs
To measure validation accuracy, we will split off a random 10 percent of your training data set
Executing op RandomStandardNormal in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/r

In [18]:
ml_predictor.score(df_test, df_test.Popularity)

None


***********************************************
Advanced scoring metrics for the trained regression model on this particular dataset:

Here is the overall RMSE for these predictions:
13.293250202503145

Here is the average of the predictions:
75.64011993408204

Here is the average actual value on this validation set:
88.0

Here is the median prediction:
75.93402862548828

Here is the median actual value:
89.0

Here is the mean absolute error:
12.359880065917968

Here is the median absolute error (robust to outliers):
12.336708068847656

Here is the explained variance:
-2.150508645062311

Here is the R-squared value:
-22.251381703467096
Count of positive differences (prediction > actual):
0
Count of negative differences:
5
Average negative difference:
-12.359880065917968


***********************************************




-13.293250202503145

In [23]:
line = data.iloc[49]
line

Beats.Per.Minute     95
Energy               79
Danceability         75
Loudness..dB..       -6
Liveness              7
Valence.             61
Length.             206
Acousticness..       21
Speechiness.         12
Popularity           82
Name: 49, dtype: int64

In [24]:
predictions = ml_predictor.predict([line])
print(predictions)

86.65128326416016
